<a href="https://colab.research.google.com/github/Rdcornejov/Data-mining-Tools/blob/main/Semana5_1_teoria_CursoIngenieriaCaracteristicas_Qu%C3%A9_es_la_ingenier%C3%ADa_de_caracter%C3%ADsticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fe-course-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F933090%2F1828856%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240913%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240913T014127Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9d7fe19e719f817faa5a016f0479a548fb287b04053db3913f3350955cf528d0a2f952b79822bb8567b15c9cdfbbb3c3a8e5685e96205ff32cdfb21834f3be063fc54abad4594ff5be9d5ae99ae253687f1a5365fe4bd9fcfc0bbbb230914e35eb5bfa8e2c4d44bd8640c383538c6fe37854afc66f5e706fe79f55138afd4bae92527f818fb9e5b19fb09648636cb419c055f6edc2dad3a2bcd219bfe4b7dd9344795e5ccf68df4740741bb9f0f1d5e04a81d7389630a75709f56802cf99ec4bb379589655f5c5422356932057c885ea9f2b61b9a78d87141856eea80933fb4a4ebd33c86d2a1731109dc8f06d891b76095b8fcb4d2fd2124b5a98114e842b64'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 27023163 bytes downloaded
Downloaded and uncompressed: fe-course-data
Data source import complete.


# ¡Bienvenido a Feature Engineering! #
En este curso, aprenderá sobre uno de los pasos más importantes para crear un excelente modelo de aprendizaje automático: la ingeniería de características .

Aprenderá a:

- Determinar qué características son las más importantes con información mutua
- Inventar nuevas características en varios dominios de problemas del mundo real
- Codificar categóricos de alta cardinalidad con una codificación de destino
- Crear funciones de segmentación con agrupamiento de k-medias
- Descomponer la variación de un conjunto de datos en características con análisis de componentes principales

Los ejercicios prácticos dan lugar a un cuaderno completo en el que se aplican todas estas técnicas para presentar una propuesta al concurso Introducción a los precios de la vivienda . Después de completar este curso, tendrá varias ideas que podrá utilizar para mejorar aún más su desempeño.

¿Estás listo? ¡Vamos allá!

El objetivo de la ingeniería de características es simplemente hacer que sus datos se adapten mejor al problema en cuestión.

Consideremos las medidas de "temperatura aparente", como el índice de calor y la sensación térmica. Estas magnitudes intentan medir la temperatura percibida por los humanos en función de la temperatura del aire, la humedad y la velocidad del viento, cosas que podemos medir directamente. Podríamos pensar en una temperatura aparente como el resultado de una especie de ingeniería de características, un intento de hacer que los datos observados sean más relevantes para lo que realmente nos importa: ¡cómo se siente realmente afuera!

Puede realizar ingeniería de características para:

mejorar el rendimiento predictivo de un modelo
reducir las necesidades computacionales o de datos
Mejorar la interpretabilidad de los resultados
Un principio rector de la ingeniería de características
Para que una característica sea útil, debe tener una relación con el objetivo que el modelo pueda aprender. Los modelos lineales, por ejemplo, solo pueden aprender relaciones lineales. Por lo tanto, cuando se utiliza un modelo lineal, el objetivo es transformar las características para que su relación con el objetivo sea lineal.

La idea clave aquí es que una transformación que se aplica a una característica se convierte, en esencia, en una parte del modelo en sí. Digamos que estás tratando de predecir la relación Pricede parcelas cuadradas de tierra a partir de la Lengthde un lado. Ajustar un modelo lineal directamente a Lengthla relación no es lineal.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/5D1z24N.png" width=300, alt="A scatterplot of Length along the x-axis and Price along the y-axis, the points increasing in a curve, with a poorly-fitting line superimposed.">
<figcaption style="textalign: center; font-style: italic"><center>Un modelo lineal se ajusta mal si solo se tiene como característica la longitud.
</center></figcaption>
</figure>

Sin embargo, si elevamos al cuadrado la Lengthcaracterística para obtener 'Area', creamos una relación lineal. Al agregar Areaal conjunto de características, este modelo lineal ahora puede ajustarse a una parábola. En otras palabras, elevar al cuadrado una característica le dio al modelo lineal la capacidad de ajustar características al cuadrado.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BLRsYOK.png" width=600, alt="Left: Area now on the x-axis. The points increasing in a linear shape, with a well-fitting line superimposed. Right: Length on the x-axis now. The points increase in a curve as before, and a well-fitting curve is superimposed.">
<figcaption style="textalign: center; font-style: italic"><center><strong>Izquierda:</strong> El ajuste al área es mucho mejor <strong>Derecha:</strong> Esto también mejora el ajuste a la longitud.
</center></figcaption>
</figure>

Esto debería mostrarle por qué puede haber un retorno tan alto del tiempo invertido en la ingeniería de características. Cualquier relación que su modelo no pueda aprender, usted puede proporcionársela a través de transformaciones. A medida que desarrolla su conjunto de características, piense en qué información podría usar su modelo para lograr su mejor rendimiento.

# Ejemplo - Formulaciones concretas #

Para ilustrar estas ideas, veremos cómo agregar algunas características sintéticas a un conjunto de datos puede mejorar el rendimiento predictivo de un modelo de bosque aleatorio (random forest)

The [*Concrete*](https://www.kaggle.com/sinamhd9/concrete-comprehensive-strength)  este dataset contiene una variedad de formulaciones de hormigón y la resistencia a la compresión del producto resultante , que es una medida de cuánta carga puede soportar ese tipo de hormigón. La tarea de este conjunto de datos es predecir la resistencia a la compresión de un hormigón dada su formulación.


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

df = pd.read_csv("../input/fe-course-data/concrete.csv")
df.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CompressiveStrength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Aquí se pueden ver los distintos ingredientes que se utilizan en cada variedad de hormigón. En un momento veremos cómo la adición de algunas características sintéticas adicionales derivadas de estos puede ayudar a un modelo a aprender relaciones importantes entre ellos.

Primero estableceremos una línea de base entrenando el modelo en el conjunto de datos no aumentado. Esto nos ayudará a determinar si nuestras nuevas funciones son realmente útiles.

Establecer valores de referencia como este es una buena práctica al comienzo del proceso de ingeniería de características. Una puntuación de referencia puede ayudarle a decidir si vale la pena conservar las nuevas características o si debería descartarlas y, posiblemente, probar algo diferente.

In [3]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Train and score baseline model
baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

MAE Baseline Score: 8.232


Si alguna vez cocinas en casa, es posible que sepas que la proporción de los ingredientes de una receta suele ser un mejor predictor de cómo resultará la receta que sus cantidades absolutas. Podríamos razonar entonces que las proporciones de las características anteriores serían un buen predictor de 'CompressiveStrength'.

La celda a continuación agrega tres nuevas características de relación al conjunto de datos.

In [4]:
X = df.copy()
y = X.pop("CompressiveStrength")

# Create synthetic features
X["FCRatio"] = X["FineAggregate"] / X["CoarseAggregate"]
X["AggCmtRatio"] = (X["CoarseAggregate"] + X["FineAggregate"]) / X["Cement"]
X["WtrCmtRatio"] = X["Water"] / X["Cement"]

# Train and score model on dataset with additional ratio features
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

MAE Score with Ratio Features: 7.948


Y, efectivamente, el rendimiento mejoró. Esto es una prueba de que estas nuevas funciones de relación expusieron información importante al modelo que antes no detectaba.

# Continue #

We've seen that engineering new features can improve model performance.  But how do you identify features in the dataset that might be useful to combine?  [**Discover useful features**](https://www.kaggle.com/ryanholbrook/mutual-information) with mutual information.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/feature-engineering/discussion) to chat with other learners.*